# Breakthrough curve for a geothermal doublet
## New analytical solution combining Schulz (1987) and Charbeneau (2000)

Interactive version. Run the notebook to interact with the analytical solution.

In [3]:
import numpy as np
from scipy.special import erfc
import matplotlib.pyplot as plt

def plot_ana(a, M, Q, phi, rho_F, c_F, rho_S, c_S, lambda_S):
    """
    Plot the analytical solution for heat transport in a doublet system.
    Parameters:
        a (float): Half the distance between wells (m)
        M (float): Aquifer thickness (m)
        Q (float): Injection/extraction rate (m³/s)
        phi (float): Porosity (-)
        rho_F (float): Fluid density (kg/m³)
        c_F (float): Fluid heat capacity (J/kg/K)
        rho_S (float): Solid matrix density (kg/m³)
        c_S (float): Solid matrix heat capacity (J/kg/K)
        lambda_S (float): Thermal conductivity (W/m/K)
    """
    a2s = 365*24*3600   # Conversion from seconds to years

    rho_A_c_A = phi * rho_F * c_F + (1 - phi) * rho_S * c_S
    G = rho_A_c_A / (rho_F * c_F)
    H = np.sqrt(lambda_S*rho_S*c_S) / (M * rho_F * c_F)
    tb = G * 4 * np.pi * M * a * a / (3 * Q) / a2s

    phi1 = np.linspace(1e-8, np.pi, 360)
    F = 1 - phi1/np.pi

    tau_array = 4 * np.pi * M * a * a / Q * (1 - np.pi * F / np.tan(np.pi * F)) / (np.sin(np.pi * F) ** 2) / a2s

    time = np.linspace(1e-8, 100, 100)
    T = np.zeros_like(time)

    for tau in tau_array:
        U = np.heaviside(time-G*tau, 1)
        ind = U > 0
        abc = erfc(H*tau*np.sqrt(a2s)/np.sqrt(time[ind]-G*tau))
        T[ind] = T[ind] + U[ind]*abc

    plt.plot(time, T/len(tau_array), '--', label="analytical solution")
    plt.plot([tb, tb], [0, 1], ':', label="breakthrough time")

    plt.xlabel("time (a)")
    plt.ylabel("temperature difference $(T - T_0) / (T_i - T_0)$ (-)")
    plt.xlim((0, 100))

    plt.legend()
    plt.show()

In [4]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

from ipywidgets import interactive_output, FloatSlider, VBox, HBox
from IPython.display import display

# Use Unicode symbols for slider descriptions
keys = ["a", "M", "Q", "phi", "rho_F", "c_F", "rho_S", "c_S", "lambda_S"]
des = ["a", "M", "Q", "φ", "ρ_F", "c_F", "ρ_S", "c_S", "λ"]

value = [450, 30, 0.03, 0.4, 1500, 4200, 1900, 850, 3.0]
vmin = [100, 10, 0.01, 0.1, 1000, 4000, 1500, 700, 1e-8] 
vmax = [1000, 50, 0.05, 0.4, 1500, 4400, 2100, 1000, 4.0]
step = [10, 2, 0.001, 0.01, 10, 10, 10, 10, 0.1]

sliders = {k: FloatSlider(value=v, min=v0, max=v1, step=s, description=d) for v, v0, v1, s, d, k in zip(value, vmin, vmax, step, des, keys)}

# Create the interactive plot
ui = VBox(list(sliders.values()))
out = interactive_output(plot_ana, sliders)
display(HBox([out, ui]))